In [1]:
import sys
import os
import numpy as np
import time
from imp import reload

In [2]:
# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")

new directory is: C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE


# Test 1: DAQ Controllers


In [3]:
from L1 import DAQControllers

## National Instruments DAQ

Tests:
1. Initialize the DAQ
2. Set an analog output Voltage
3. Read an analog input voltage
4. Set an digitalouput voltage

In [4]:
ni = DAQControllers.NiDaq()

def print_data(data,time,channels,*args):
    print(f"Total samples: {len(data[0])}, average: {np.mean(data)} V ")
    
ni.add_callback(print_data, ['ai0'], "wave",())
print("Initialize complete...")

Initialize complete...


In [5]:
ni.add_analog_output('ao0')
ni.set_channel_voltage('ao0',0.05)
ni.start_voltage()
print("Analog output voltage set, measure channel ao0. \nMeter should read 50 mV")

Analog output voltage set, measure channel ao0. 
Meter should read 50 mV


In [6]:
ni.add_analog_input('ai0')
ni.set_sampling_frequency(1000)
ni.start_measurement()
time.sleep(.1)
ni.stop_measurement()
print("\nThere should be 100 samples read in 0.1 s \n")

Total samples: 100, average: -5.310797896931989 V 
There should be 100 samples read in 0.1 s 




In [7]:
ni.add_do_channel('p0.0')
ni.set_do_channel('p0.0', True)
ni.update_do_channels()
print("Digital output on Port0/line0 should read digital High")

Digital output on Port0/line0 should read digital High


# Test 2: Linear Motion Utilities

tests:
1. Home to Zero position
2. Read and set position
3. Stop movement

In [6]:
from L1 import Controllers
from L2 import ZControl
reload(ZControl)
reload(Controllers)

C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py


<module 'L1.Controllers' from 'C:\\Users\\NikonTE300CE\\Desktop\\Barracuda\\AutomatedCE\\L1\\Controllers.py'>

In [3]:
unit_scalars = {'mm':1, "um":1000}
# Check Stop Capabilities
def check_z_stop(z, units='mm'):
    pos = z.read_z()
    travel = 4 * unit_scalars[units]
    z.set_rel_z(travel)
    time.sleep(0.2)
    z.stop()
    z.read_z()

    print(f"Motor should stop moving before it reaches {pos+travel} {units}")
    print(f"Final position: {z.read_z()} {units}")

In [4]:
# Check Homing Capability
def check_z_homing(z):
    z.homing()
    print("Thorlabs should be at the 0 mm height, powerstep at 29.75 mm")

In [5]:
# Check Movment capabilities 
def check_z_movement(z, units = 'mm'):
    
    travel = 0.05 * unit_scalars[units]
    pos = z.read_z()
    z.set_rel_z(travel)
    time.sleep(2)
    final = z.read_z()

    print(f"Initial position was {pos} {units}, after move: {final} {units}")
    assert pos != final, print("Check Z Stage, it appears it did not move")

### Thorlab Kinesis Motor

In [39]:
ser_num = '49125264'
z = ZControl.KinesisZ(ser_num,'zstage')
z.startup()
print("Thorlabs Initialized...")


Thorlabs Initialized...


In [68]:
check_z_homing(z)

TypeError: check_z_homing() takes 1 positional argument but 2 were given

In [66]:
check_z_movement(z, 'mm')

Initial position was 1.0 um, after move: 51.0 um


In [70]:
check_z_stop(z, 'mm')

Motor should stop moving before it reaches 4051.0 um
Final position: 612.6 um


In [71]:
z.shutdown()

### Arduino PowerStep Motion

In [34]:
ctl = Controllers.ArduinoController("COM8")
ctl.open()
z = ZControl.ArduinoZ(ctl, 'zstage')
z.startup()

In [35]:
check_z_homing(z)

Thorlabs should be at the 0 mm height, powerstep at 29.75 mm


In [36]:
z.set_z(25)
z.wait_for_move()
check_z_movement(z)

Initial position was 25.0 mm, after move: 25.05 mm


In [37]:
check_z_stop(z)

Motor should stop moving before it reaches 27.05
Final position: 26.887999999999998


In [38]:
z.shutdown()
ctl.close()

### Prior Z Motion

In [7]:
ctl = Controllers.PriorController("COM9")
ctl.open()
z = ZControl.PriorZ(ctl,'zstage')
z.startup()
z.min_z = -100000
z.max_z = 100000

In [58]:
check_z_homing(z)

Thorlabs should be at the 0 mm height, powerstep at 29.75 mm


In [8]:
check_z_movement(z, 'um')

Initial position was 4051.0 um, after move: 4101.0 um


In [9]:
check_z_stop(z, 'um')

Motor should stop moving before it reaches 8101.0 um
Final position: 4722.1 um


In [11]:
z.shutdown()
ctl.close()

# Test 3: XY Stage 

tests:
1. check acceleration and velocity settings
2. check movement
3. check stop


In [108]:
from L1 import Controllers
from L2 import XYControl
reload(XYControl)
reload(Controllers)

<module 'L1.Controllers' from 'C:\\Users\\NikonTE300CE\\Desktop\\Barracuda\\AutomatedCE\\L1\\Controllers.py'>

In [120]:
def check_xy_motor_params(xy):
    a = xy.get_acceleration()
    v = xy.get_velocity()
    print(f"{a} mm/s/s, and {v} mm/s should be the stage parameters")
    
def check_xy_movement(xy):
    pos = xy.read_xy()
    xy.set_rel_xy([0.05,0.05])
    time.sleep(.1)
    final = xy.read_xy()
    print("Stage should move 50 microns towards the back of the scope"
         "and 50 microns towards the right.")
    print(f"Original pos {pos}, \nfinal pos {final} [mm,mm]")
    
    
def check_xy_stop(xy):
    input("Move stage to center, press enter when ready")
    pos = xy.read_xy()
    xy.set_rel_xy([5,5])
    xy.stop()
    final = xy.read_xy()
    xy.stop()
    print("Stage should stop before moving 5mm in x and y\n"
         f"Original pos: {pos}, final pos: {final}")
    if final[0]-pos[0] >=4.95 or final[1] - pos[1]>=4.95:
        print("Warning: Stage does not stop when asked!")
    else:
        print("Stage stopped before 5mm")
    

### Prior XY Stage

In [109]:
ctl = Controllers.PriorController('COM9')
ctl.open()
xy = XYControl.PriorXY(ctl,'xystage')
xy.startup()

In [51]:
check_xy_motor_params(xy)

114.2 mm/s/s, and 19.0 mm/s should be the stage parameters


In [121]:
check_xy_movement(xy)


Stage should move 50 microns towards the back of the scopeand 50 microns towards the right.
Original pos [6.679, -15.758], 
final pos [6.731, -15.706] [mm,mm]


In [119]:
check_xy_stop(xy)

Move stage to center, press enter when ready
Stage should stop before moving 5mm in x and yOriginal pos: [3.524, -18.93], final pos: [6.679, -15.758]
Stage stopped before 5mm


In [107]:
xy.shutdown()
ctl.close()

# Test 4: Capillary Pressure Control
test:
1. Turn Pressure On
2. Turn Vacuum On


In [129]:
from L1 import Controllers
from L2 import PressureControl
reload(PressureControl)
reload(Controllers)

<module 'L1.Controllers' from 'C:\\Users\\NikonTE300CE\\Desktop\\Barracuda\\AutomatedCE\\L1\\Controllers.py'>

In [141]:
def check_p_pressure(p):
    p.rinse_pressure()
    input("The line should be pressurized, verify and press enter to stop")
    p.release()
    return

def check_p_vacuum(p):
    p.rinse_vacuum()
    input("The line should be under vacuum, verify and press enter to stop")
    p.release()
    return


In [131]:
ctl = Controllers.ArduinoController('COM8')
ctl.open()
p = PressureControl.ArduinoPressure(ctl,'outletpressure')
p.startup()

In [140]:
check_p_pressure(p)

The line should be pressurized, verify and press enter to stop


In [142]:
check_p_vacuum(p)

The line should be under vacuum, verify and press enter to stop


In [143]:
p.shutdown()
ctl.close()